In [ ]:
%pip install -U "autogen-agentchat"
%pip install "autogen-ext[openai]"
%pip install "autogen-ext[azure]"

# LLMS

Con OpenAI

```python
from autogen_ext.models.openai import OpenAIChatCompletionClient

openai_model_client = OpenAIChatCompletionClient(
    model="gpt-4o-2024-08-06",
    # api_key="sk-...", # Optional if you have an OPENAI_API_KEY environment variable set.
)
```


Con Azure OpenAI

In [ ]:
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient
from azure.identity import DefaultAzureCredential, get_bearer_token_provider
from autogen_core.models import UserMessage
import os


az_model_client = AzureOpenAIChatCompletionClient(
    azure_deployment= "gpt-4o-mini", 
    model="gpt-4o-mini",  
    api_version=os.environ["OPENAI_API_VERSION"],
    api_key=os.environ["AZURE_OPENAI_API_KEY"],
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    

)

result = await az_model_client.create([UserMessage(content="What is the capital of France?", source="user")])
print(result)

AttributeError: 'str' object has no attribute 'content'

In [ ]:
from autogen_ext.models.openai import OpenAIChatCompletionClient
import os
from autogen_core.models import UserMessage


openai_model_client = OpenAIChatCompletionClient(
    model="gpt-4o-2024-08-06",
api_key= os.getenv("OPENAI_API_KEY")
)

result = await openai_model_client.create([UserMessage(content="What is the capital of France?", source="user")])
print(result)

finish_reason='stop' content='The capital of France is Paris.' usage=RequestUsage(prompt_tokens=15, completion_tokens=8) cached=False logprobs=None thought=None


# Mensajes 

En **AutoGen AgentChat**, los mensajes son la base de la comunicación entre agentes, orquestadores y aplicaciones. Se dividen en dos grandes categorías:

1. **Mensajes entre agentes** (Agent-Agent Messages)
2. **Eventos y mensajes internos de un agente** (Internal Events)

### **1. Mensajes entre agentes (Agent-Agent Messages)**

Estos mensajes permiten la comunicación entre agentes y pertenecen al tipo `ChatMessage`. Pueden ser:

- **Mensajes de texto** (`TextMessage`)
- **Mensajes multimodales** (`MultiModalMessage`), que pueden incluir imágenes.

#### **Ejemplo: Creación de un mensaje de texto**

Para enviar un mensaje de texto entre agentes:

In [4]:
from autogen_agentchat.messages import TextMessage

text_message = TextMessage(content="Hola, mundo!", source="User")
print(text_message)


source='User' models_usage=None content='Hola, mundo!' type='TextMessage'


**Ejemplo: Creación de un mensaje multimodal**

Para enviar un mensaje con una imagen:

In [5]:
from io import BytesIO
import requests
from autogen_agentchat.messages import MultiModalMessage
from autogen_core import Image as AGImage
from PIL import Image

# Cargar una imagen desde una URL
pil_image = Image.open(BytesIO(requests.get("https://picsum.photos/300/200").content))
img = AGImage(pil_image)

# Crear un mensaje multimodal con texto e imagen
multi_modal_message = MultiModalMessage(content=["¿Puedes describir esta imagen?", img], source="User")
print(multi_modal_message)


source='User' models_usage=None content=['¿Puedes describir esta imagen?', <autogen_core._image.Image object at 0x00000159448792B0>] type='MultiModalMessage'


### **2. Eventos internos del agente (Internal Events)**

Además de los mensajes entre agentes, existen **eventos internos** que representan acciones dentro de un agente. Estos eventos pertenecen al tipo `AgentEvent`.

Algunos ejemplos de eventos internos:

- **`ToolCallRequestEvent`** → Indica que un agente ha solicitado usar una herramienta.
- **`ToolCallExecutionEvent`** → Contiene los resultados de la ejecución de una herramienta.

Los eventos internos se generan automáticamente y se almacenan en el campo `inner_messages` dentro de la respuesta del método `on_messages()`.

# **Agents**

AutoGen proporciona un conjunto de agentes predefinidos que pueden responder a mensajes de diferentes maneras. Todos los agentes comparten ciertos atributos y métodos clave:

## **Atributos y Métodos Comunes**

- `name`: Nombre único del agente.
- `description`: Descripción en texto del agente.
- `on_messages()`: Recibe una secuencia de mensajes y devuelve una respuesta. **Modifica el estado del agente**, por lo que no se debe llamar con el historial completo de mensajes.
- `on_messages_stream()`: Igual que `on_messages()`, pero devuelve un iterador de eventos `AgentEvent` o `ChatMessage`, seguido de una `Response` final.
- `on_reset()`: Restablece el agente a su estado inicial.
- `run()` y `run_stream()`: Métodos de conveniencia que llaman a `on_messages()` y `on_messages_stream()` respectivamente, pero con la misma interfaz que los equipos de agentes.

## **Ejemplo: Creando un Assistant Agent**

`AssistantAgent` es un agente predefinido que usa un modelo de lenguaje y puede ejecutar herramientas.

### **Paso 1: Importar las librerías necesarias**

In [7]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import TextMessage
from autogen_agentchat.ui import Console
from autogen_core import CancellationToken
from autogen_ext.models.openai import OpenAIChatCompletionClient

### **Paso 2: Definir una herramienta personalizada**


In [8]:
# Herramienta para buscar información en la web.
async def web_search(query: str) -> str:
    """Encuentra información en la web"""
    return "AutoGen es un framework de programación para construir aplicaciones multiagente."

### **Paso 3: Crear el agente con un modelo de OpenAI**

In [9]:
agent = AssistantAgent(
    name="assistant",
    model_client=openai_model_client,
    tools=[web_search],  # Asigna la herramienta de búsqueda
    system_message="Usa herramientas para resolver tareas.",
)

## **Obteniendo Respuestas**

El método `on_messages()` permite obtener respuestas a los mensajes enviados al agente.

### **Ejemplo: Obtener una respuesta del asistente**

In [10]:
async def assistant_run() -> None:
    response = await agent.on_messages(
        [TextMessage(content="Encuentra información sobre AutoGen", source="user")],
        cancellation_token=CancellationToken(),
    )
    print(response.inner_messages)  # Mensajes internos del agente
    print(response.chat_message)  # Respuesta final del agente

# Para ejecutar en un script:
# asyncio.run(assistant_run())
await assistant_run()


[ToolCallRequestEvent(source='assistant', models_usage=RequestUsage(prompt_tokens=64, completion_tokens=16), content=[FunctionCall(id='call_rgqHzmkfi5saRWNFKscn1717', arguments='{"query":"AutoGen"}', name='web_search')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(source='assistant', models_usage=None, content=[FunctionExecutionResult(content='AutoGen es un framework de programación para construir aplicaciones multiagente.', call_id='call_rgqHzmkfi5saRWNFKscn1717')], type='ToolCallExecutionEvent')]
source='assistant' models_usage=None content='AutoGen es un framework de programación para construir aplicaciones multiagente.' type='ToolCallSummaryMessage'



## **Respuestas en Streaming**

Podemos recibir respuestas en tiempo real con `on_messages_stream()`. También se puede utilizar `Console` para visualizar los mensajes en la consola.

### **Ejemplo: Respuesta en streaming**

In [11]:
async def assistant_run_stream() -> None:
    await Console(
        agent.on_messages_stream(
            [TextMessage(content="Encuentra información sobre AutoGen", source="user")],
            cancellation_token=CancellationToken(),
        )
    )
# Para ejecutar en un script:
# asyncio.run(assistant_run_stream())
await assistant_run_stream()


---------- assistant ----------
AutoGen es un framework de programación diseñado para construir aplicaciones multiagente. Los frameworks multiagentes permiten crear aplicaciones donde múltiples agentes inteligentes pueden interactuar y trabajar en conjunto para lograr objetivos complejos de manera autónoma y organizada. Sin embargo, podría haber más detalles y contextos específicos sobre AutoGen dependiendo del ámbito particular (tecnológico, industrial, etc.) en el que se esté utilizando o discutiendo. Si necesitas información más detallada, puedo realizar una búsqueda más profunda o específica.


## **Otros Agentes Predefinidos**

AutoGen también proporciona otros agentes con diferentes funcionalidades:

- **UserProxyAgent**: Devuelve la entrada del usuario como respuesta.
- **CodeExecutorAgent**: Puede ejecutar código.
- **OpenAIAssistantAgent**: Conexión con un OpenAI Assistant y herramientas personalizadas.
- **MultimodalWebSurfer**: Busca y visita páginas web.
- **FileSurfer**: Explora archivos locales.
- **VideoSurfer**: Analiza información de videos.

--- 
## **Uso de Herramientas**

Los modelos de lenguaje por sí solos están limitados a generar texto. Sin embargo, **AutoGen permite que los agentes usen herramientas externas**, como llamadas a APIs o bases de datos.

📌 **Tool Calling / Function Calling**: Los modelos modernos pueden aceptar herramientas definidas por el usuario y generar un mensaje de llamada a una herramienta.

### **Ejemplo: Agente con herramientas de LangChain**

Podemos integrar herramientas de LangChain usando `LangChainToolAdapter`.

In [13]:
import pandas as pd
from autogen_ext.tools.langchain import LangChainToolAdapter
from langchain_experimental.tools.python.tool import PythonAstREPLTool

# Cargar un dataset de Titanic en Pandas
df = pd.read_csv("https://raw.githubusercontent.com/pandas-dev/pandas/main/doc/data/titanic.csv")

# Crear herramienta de análisis con LangChain
tool = LangChainToolAdapter(PythonAstREPLTool(locals={"df": df}))

# Configurar el agente con herramientas de LangChain
agent = AssistantAgent(
    "assistant",
    tools=[tool],
    model_client=openai_model_client,
    system_message="Usa la variable `df` para acceder al dataset.",
)

# Consultar la edad promedio de los pasajeros
await Console(
    agent.on_messages_stream(
        [TextMessage(content="¿Cuál es la edad promedio de los pasajeros?", source="user")], 
        CancellationToken()
    )
)


---------- assistant ----------
[FunctionCall(id='call_2sKC2cshNTlFSpKuwpRwsTHD', arguments='{"query":"df[\'Age\'].mean()"}', name='python_repl_ast')]
---------- assistant ----------
[FunctionExecutionResult(content='29.69911764705882', call_id='call_2sKC2cshNTlFSpKuwpRwsTHD')]
---------- assistant ----------
29.69911764705882


Response(chat_message=ToolCallSummaryMessage(source='assistant', models_usage=None, content='29.69911764705882', type='ToolCallSummaryMessage'), inner_messages=[ToolCallRequestEvent(source='assistant', models_usage=RequestUsage(prompt_tokens=114, completion_tokens=22), content=[FunctionCall(id='call_2sKC2cshNTlFSpKuwpRwsTHD', arguments='{"query":"df[\'Age\'].mean()"}', name='python_repl_ast')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(source='assistant', models_usage=None, content=[FunctionExecutionResult(content='29.69911764705882', call_id='call_2sKC2cshNTlFSpKuwpRwsTHD')], type='ToolCallExecutionEvent')])

## **Llamadas a Herramientas en Paralelo**

Algunos modelos permiten ejecutar múltiples herramientas a la vez.

📌 Para desactivar esta opción en `OpenAIChatCompletionClient`, establece `parallel_tool_calls=False`:

In [ ]:
model_client_no_parallel_tool_call = OpenAIChatCompletionClient(
    model="gpt-4o",
    parallel_tool_calls=False,  # Desactiva llamadas paralelas
)

## **Salida Estructurada con JSON**

Podemos hacer que el modelo devuelva respuestas estructuradas en JSON usando `Pydantic`.

### **Ejemplo: Respuesta con formato JSON**

In [14]:
from typing import Literal
from pydantic import BaseModel

# Modelo de respuesta estructurada
class AgentResponse(BaseModel):
    thoughts: str
    response: Literal["happy", "sad", "neutral"]

# Crear agente con salida estructurada
model_client = OpenAIChatCompletionClient(
    model="gpt-4o",
    response_format=AgentResponse,  # Especifica el formato de respuesta
    api_key=os.getenv("OPENAI_API_KEY"),
)
agent = AssistantAgent(
    "assistant",
    model_client=model_client,
    system_message="Categoriza el input como 'happy', 'sad' o 'neutral'.",
)

await Console(agent.run_stream(task="I am happy."))


---------- user ----------
I am happy.
---------- assistant ----------
{"thoughts":"The user explicitly states their emotion using the word 'happy', which clearly indicates their emotional state.","response":"happy"}


TaskResult(messages=[TextMessage(source='user', models_usage=None, content='I am happy.', type='TextMessage'), TextMessage(source='assistant', models_usage=RequestUsage(prompt_tokens=88, completion_tokens=28), content='{"thoughts":"The user explicitly states their emotion using the word \'happy\', which clearly indicates their emotional state.","response":"happy"}', type='TextMessage')], stop_reason=None)